In [1]:
from data_preparation import DataPreparator
from apply_clustering import ClusteringApplier

In [7]:
import os.path
from itertools import chain, combinations
from typing import Dict, List, Tuple, Iterable, Any

import numpy as np
import pandas as pd
from gapstatistics.gapstatistics import GapStatistics
from clustering import AgglomerativeClusteringWrapper as AgglomerativeClustering
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from scipy.spatial.distance import jensenshannon, correlation, euclidean
from tqdm.contrib.concurrent import thread_map
import seaborn as sns

import config

In [8]:
df_explainable: pd.DataFrame = ClusteringApplier.read_explaining_features()
df_observable_distances: pd.DataFrame = pd.read_excel(
    f"{config.OUTPUT_FOLDER_BASE}observables/{config.DATASET_NAME}-distance-normalized-matrix-{config.DISTANCE_MEASURE_FINGERPRINT}-{config.NUMBER_OBSERVABLE_PATTERNS}.xlsx",
    index_col=0
)
features: List[str] = list(config.EXPLAINING_FEATURE_NAMES.keys())

# Ensure that both datasets contain the same indices
valid_indices = np.intersect1d(df_explainable.index, df_observable_distances.index)
df_explainable = df_explainable.loc[valid_indices]
df_observable_distances = df_observable_distances.loc[valid_indices]

In [17]:
assert df_observable_distances.index.to_list() == df_observable_distances.columns.to_list(), 'Something went wrong: Expected df_observable_distances index to be equal to its columns.'

In [19]:
df_explainable

,Age,Sex,Zoo,Stable,Genus_ID,Family_ID,Order_ID,SH,Weight,Habitat
Individual,,,,,,,,,,
A.nas_01,0.998484,0.000580,0.100058,0.998765,0.172335,0.998493,0.499652,0.601253,0.040563,0.998516
A.nas_02,0.999210,1.000036,0.100573,-0.000347,0.171117,0.998302,0.499261,0.600382,0.040902,0.999630
A.nas_03,0.998590,0.999640,0.099918,0.001408,0.168244,0.997768,0.501590,0.599119,0.039904,0.998778
A.nas_05,1.000408,0.999080,0.448777,0.998671,0.170826,0.999221,0.502037,0.600641,0.040503,0.999217
A.nas_06,1.000104,0.999871,0.452889,0.999723,0.170719,0.999238,0.498438,0.599065,0.040647,0.997738
...,...,...,...,...,...,...,...,...,...,...
T.spe_01,1.001301,-0.000548,0.549917,1.000873,0.382095,0.999667,0.499275,0.520286,0.033298,-0.000350
T.str_01,1.000954,0.000332,0.501687,0.998202,0.385288,1.000324,0.500775,0.792802,0.112167,0.333072
T.str_02,1.000768,-0.000142,0.599987,1.001765,0.383631,0.999242,0.500329,0.791013,0.109779,0.334542
